# Milestone 1
#### Group members: Robin Robinson, Anna Midgley, Nora Hallqvist, Sebastian Weisshaar

# Introduction 

There are four main approaches to computing the derivative which are namely manual, symbolic, numerical, or automatic differentiation. Manual differentiation is the calculation of the derivative expression by hand and its coding. It is time-consuming and prone to error especially as systems become more complex. Numerical differentiation is the finite difference approximation of the derivative using values of the original function evaluated at points. Its advantage is that it is easy to implement whilst its disadvantage is truncation and round-off errors. Symbolic differentiation uses packages such as Mathematica, to produce the exact derivative expression as its output. It addresses the weaknesses of manual and numerical differentiation; however, it frequently results in “expression swell”. Expression swell is the phenomenon of much larger representation of a derivative as opposed to representation of the original function. Symbolic differentiation does not consider the fact that many sub-expressions in the different derivative expressions are common. This leads to inefficiency in its calculation. Symbolic and manual differentiation require the model to be defined as closed-form expressions which limits the expressivity of the model as it cannot use branches or loops. The fourth technique, automatic differentiation (AD), obtains the exact result of the derivative by breaking down the expression into elementary arithmetic operations and elementary functions and applying the chain rule to the derivatives of these operations.

# Background

## **Automatic Differentiation**

The key idea of Automatic Differentation is to decompose the calculations into elementary functions and  evaluate the functions derivative by combining each elementary function's derivative by using the chain rule.


## **Chain Rule**

The **chain rule** is a method to differentiate composite functions. This is especially useful in automatic differentation when extended to the multivariate case.

For example, suppose we have a multivariate function $f(u_{1}(t),u_{2}(t))$ and we want to evaluate the derivative $f$ with respect $t$. Then we have the following expression: 

$$ \frac{\partial f(u_{1}(t),u_{2}(t))}{\partial t} = \frac{\partial f}{\partial u_{1}}\frac{\partial u_{1}}{\partial t} + \frac{\partial f}{\partial u_{2}}\frac{\partial u_{2}}{\partial t}$$

The above can also be represented in vector notation. Replacing $t$ with $x \in R^{m}$, the chain rule can more elegantly be written in terms of the differential vector operator $\nabla$ with respect to the vector $x$.

$$\nabla_{x}f = \frac{\partial f}{\partial u_{1}}\nabla_{x}u_{1}  + \frac{\partial f}{\partial u_{2}}\nabla_{x}u_{2} $$

where $f = f(u_{1}(x_{1},...,x_{m}),u_{2}(x_{1},...,x_{m}))$

Automatic differentiation commonly involves several dependent variables (i.e intermediate steps), hence we need to generalize the chain rule to support a function $f$ of $n$ other functions (i.e $f((u_{1}(x_{1},...,x_{m}),u_{2}(x_{1},..,x_{m}),..,u_{n}(x_{1},..,x_{m}))$)

Now the gradient of $f$ is instead given by: 

$$\nabla_{x}f = \sum_{i=1}^{n} \frac{\partial f}{\partial u_{i}(x)} \nabla_{x}u_{i}(x)$$

where $x \in R^{m}$


## **Compuational Graph**

Automatic differentiation can be represented as a graph structure. This is done by decomposing the function into its primitive operations (e.g binary arithmic operations and unary operations). The computation graph is then simply constructed by representing each elementary operation as a node, and connecting each node by an edge. 

By convention the nodes are represented by the following variables:  
- Input variables : $v_{i-m} = x_{i}$ , $i = 1,...,m$
 - Intermediate variables: $v_{i}$ , $i = 1,..,n$

An example of a computational graph is shown below for the function $f(x)=sin(x_1x_2) + x_2$,

```mermaid
graph LR;
	x_1 --> v_-1;
	x_2 --> v_0;
    v_-1 -- multiply --> v_1;
    v_0 -- multiply --> v_1;
    v_1 -- sin --> v_2;
    v_2 -- add --> v_3;
    v_0 -- add --> v_3;
    v_3 --> f;
```

## **Evaluation Trace**

The evaluation trace stores the intermediate results $v_{i}$ evaluated at a given point, and includes a tangent trace, which records the directional derivative of each intermediate variable $v_{i}.$ The final derivative of the original function is calculated by combining derivatives of the intermediate results by using the chain rule.

Below is an example of an evaluation trace table using the previous example function and computational graph:  
  
| **Primal Trace**  | **$f(x)$**              |
|-------------------|-------------------------|
| $v_{-1} = x_1$    | $v_{-1} = x_1$          |
| $v_{0} = x_2$     | $v_{0} = x_2$           |
| $v_1 = v_{-1}v_0$ | $v_1 = x_1x_2$          |
| $v_2 = sin(v_1)$  | $v_2 = sin(x_1x_2)$     |
| $v_3 = v_2 +v_0$  | $v_3 = sin(x_1x_2)+x_2$ |
  
<br>

| **Tangent Trace**                                                               | $\frac{\partial f}{\partial x_1}$ --> Direction: $p = [1, 0]^T$ | $\frac{\partial f}{\partial x_2}$ --> Direction: $p = [0, 1]^T$          |
|---------------------------------------------------------------------------------|-----------------------------------------------------------------|--------------------------------------------------------------------------|
| $D_pv_{-1} = p_1$                                                               | $D_pv_{-1} = 1$                                                 | $D_pv_{-1} = 0$                                                          |
| $D_pv_{0} = p_2$                                                                | $D_pv_{0} = 0$                                                  | $D_pv_{0} = 1$                                                           |
| $D_pv_{1} = v_{-1}D_pv_{0} + v_{0}D_pv_{-1} = x_1p_2 + x_2 p_1$                 | $D_pv_{1} =x_2 p_1 = x_2$                                       | $D_pv_{1} =x_1 p_0 = x_1$                                                |
| $D_pv_{2} = cos(v_1)D_pv_{1}= cos(x_1 x_2)(x_1p_2 + x_2 p_1)$            | $D_pv_{2} = cos(x_1x_2)(x_2 p_1) = cos(x_1 x_2)(x_2)$    | $D_pv_{2} = cos(x_1 x_2)(x_1 p_2)$                                       |
| $D_pv_{3} = D_pv_{2} + D_pv_{0} = cos(x_1 x_2)(x_1p_2 + x_2 p_1) + p_2$  | $D_pv_{3} = cos(x_1 x_2)(x_2)$                                  | $D_pv_{3} = cos(x_1 x_2)(x_1 p_2) + p_2 = cos(x_1 x_2)(x_1) + 1$  |


  
## Seed vector
The methods mentioned above describe how to calculate the derivative of a function using a computational graph. However, we also have to consider the direction of the derivative. We could be interested in the derivative in the direction of a specific input variable or in a different direction. To find the derivative in the direction that we are interested in we input a seed vector into the graph. This seed vector described the direction of the desired derivative and is used in the various steps of calculating the derivative. For example, if we are interested in $\frac{\partial f(x_1,x_2)}{\partial x_1}$ we would input $[1,0]^T$ as our seed vector to indicate the direction of the derivative we are interested in. 
  
    





## Dual numbers

A dual number is given by the expression:

z = a + b $\epsilon$ such that  $\epsilon>0$ and  $\epsilon^{2}=0$


Similiar to complex numbers, dual numbers adhere to the following rules for addition and multiplication:

$z_{1} + z_{2} = (a_{1}  + a_{2})  + (b_{1} + b_{2}) \epsilon$
$z_{1} z_{2} = (a_{1} *a_{2})  + (a_{1}*b_{2} + a_{2}*b_{1}) \epsilon$


Let the real part be equal to the functions $f_{1}(x) \text{ and } f_{2}(x)$ (i.e $a_{1} = f_{1}(x) \text{ and } a_{2} = f_{2}(x) $ ) and the dual part be equal to their respective derivatives (i.e $ b_{1} = f_{1}'(x) \text{ and } b_{2} = f_{2}'(x)$). Then we get the following expressions: 

$z_{1} + z_{2} = (f_{1}  + f_{2})  + (f_{1}'(x) + f_{2}'(x)) \epsilon$
$z_{1} z_{2} = (f_{1}f_{2})  + (f_{1}f_{2}'(x) + f_{2}f_{1}'(x)) \epsilon$


By using dual numbers in forward automatic differentiation, both the primal and tangent trace can be carried as a pair. This can be seen by letting the functions represent the intermediate results in the primal trace (i.e $f_{i}(x) = v_{i}$) and the functions' derivatives be equal to the tangent trace (i.e $f_{i}'(x) = D_{p}v_{i})$.

For dual numbers to be useful in automatic differentiation, a function applied to a dual number must satisfy the chain rule. In general, any analytic function can be extended to the dual numbers by looking at the function's taylor series: 

$f(z) = f(a+b\epsilon) = f(a) + bf'(a)\epsilon$

note: all higher terms disappear since $\epsilon^{0}=0$

Applying the function $f$ on the dual number $z_{i} = v_{i} + D_{p}v_{i}\epsilon$, we see that any analytic function applied to a dual number returns another dual number:

$f(z_{i}) = f(v_{i}+ D_{p}v_{i}\epsilon) = f(v_{i}) + f'(v_{i})D_{p}v_{i}\epsilon$

From the above expression we can see that the dual part is equal to the dual part of $z_{i} \text{ times by }  f'(v_{i})$  **i.e the chain rule**






An example of how dual numbers can be used in the forward pass in automatic differentiation is shown below using the previous example. It shows how binary arithmatic operations on dual number maintain the correct value of the primal and tangent trace of the new node. This is shown for the addition of node $v_2$ to $v_0$ to become $v_3$.  

$v_3 = v_2 + v_0 = (f_2 + f_0) + (f'_2 + f'_0)\epsilon$ 

$v_3 = \left(sin(x_1x_2) + x_1\right) + \left(\cos(x_1x_2)(x_1p_2+x_2p_1) + p_1\right)\epsilon$

It should be noted that we will not use dual numbers. However, the principles we have just mentioned will be maintained in the Node class.  


# How to Use *autodiff*

The name of our package is *autodiff*. Its functionality is to provide the user with methods to undergo automatic differentiation of both univariate and multivariate functions. We envision the user should import the autodiff and functions modules. The imported unary functions should be used to define the function that is to be evaluated. This function can either be a scalar or an array. The user should define an input and seed, and these parameters are used to instantiate the AutoDiff class. The input is the point at which the derivative/function is evaluated at. The seed is the vector direction at which the derivative is computed in, commonly denoted as p. The seed vector must be the same shape as the input vector. The user can access forward or backward mode by calling a method of the AutoDiff class.

An example of how we envision the user would interact with the package is demonstrated below.

**Pseudo Code**


```Python
import autodiff as AD

from functions import sin,cos,ln,exp,tanh,...

f = lambda x: {ln(x[0]) + sin(x[0]+x[1])
input = [1,1]
seed = [1,0]
diff_f = AD.AutoDiff(f,input,seed)
forward_mode = diff_f.forward() #returns derivative value using forward mode
backward_mode = diff_f.backward() #returns derivative value using backward mode
eval = diff_f.function_value() #returns function value evaluated at input 

```






# Organizational Structure  

### Directory Structure

*Note that the directory structure is subject to change depending on how we decide to implement reverse mode. The alternative would be to create two subpackages, one for forward mode and another for reverse mode.*


|-- src/
|&emsp;|-- Autodiff/
|&emsp;|&emsp;|-- \_\_init__.py
|&emsp;|&emsp;|-- autodiff.py
|&emsp;|&emsp;\\-- functions.py
|&emsp;|-- demo
|&emsp;\\-- tests
|-- Docs/
|&emsp;\\-- README.md
|-- LICENSE
|-- pyproject.toml
\\-- README.md


### Modules
- **\_\_init__.py**: Imports all of the necessary modules

- **autodiff.py**: Contains the AutoDiff & Node class.
    - The AutoDiff class is the main interface between the user and the package. The class takes in a function, input vector, and seed vector. It provides the user the functionality to compute the derivative, in either forward or reverse mode, and the function evaluation.
    - The Node class is the main data structure which stores the information of the a specific node in the computational graph including name, value, child, parents, and adjoint. In addition the Node class overloads the binary arithmatic operations. 
    - The following is an example of how we would overload the addition operation,
    ``` Python
    def __add__(self, other):
        new_name = max(self.name, other.name) + 1
        for_deriv = self.for_deriv + other.for_deriv
        back_deriv = {self.name: 1, other.name: 1}
        new_node = Node(new_name, self.value + other.value, for_deriv=deriv, back_deriv=back_deriv, parents=[self, other])
        self.child.append(new_node)
        other.child.append(new_node)
        return new_node
    ```
    
- **functions.py**: Contains defined unary functions for dual numbers
    - This includes: trig functions, inverse trig functions, exponentials, hyperbolic functions, logistic function, logarithms and square root
    - The following is an example how to define an unary function,
    ``` Python
    def exp(node):
        new_name = node.name + 1
        for_deriv = np.exp(node.value)*node.for_deriv
        back_deriv = {node.name: np.exp(node.value)}
        new_node = Node(new_name, value, for_deriv=deriv, back_deriv=back_deriv, parents=[node])
        node.child.append(new_node)
    return new_node
    ```

### Demo and Test Suite
Both the test and demo suites will live in the source code (src) directory. The test folder will contain Python files that test the AutoDiff class. The demo folder will contain example usages of the package. 

### Package Distribution
The package will be distributed using PyPI with PEP517/518. We will add a pyproject.toml file to our project. This enables us to build our package using a PEP517 builder and distribute our package using PyPI. Other developers can install it using pip install. 


# Implementation

The table below lists all of the classes needed to implement forward mode, with their respective methods and attributes. The core data structure are Nodes which are used for both forward and reverse mode. We will implement Node first, followed by AutoDiff. After which, we will move on to implementing the extension of the project. 

| **Class**      | **AutoDiff**               | **Node**                                                                                      |   |   |
|----------------|----------------------------|------------------------------------------------------------------------------------------------------|---|---|
| **Attributes** &emsp;&emsp;| f, input, seed,       | name, value, child, parent, for_deriv, back_deriv, adjoint,                                                                                            |   |   |
| **Methods**    | function_value, forward, backward &emsp;&emsp;| \_\_add__, \_\_radd__, \_\_mul__, \_\_rmul__, \_\_truediv__, \_\_rtruediv__, \_\_pow__, \_\_neg__ |   |   |

<br>
We will not use DualNumbers. Using dual numbers as the primary data structure can only be used in forward mode. If we decide to implement reverse mode, this will result in redundancy of code, as we would need to define a different data structure.  
As mentioned earlier, for the Node class, we will overload the binary arithmatic operations. Unary functions such  as ```sin``` or ```log``` will be defined in functions.py file for the Node data structures. These functions will work for both forward and reverse mode of AD, because both rely on the Node data structure. 


We would like our package to be able to compute the derivative of both scalar and vector functions that can depend on both scalar and vector parameters. To ensure this, we have two requirements for how the user defines the function. The first is that the input of the function must be the same size as the number of parameters (m). The second is that the function that is passed to the AutoDiff must be a list or an array of functions of size of the number of outputs (n).    

For example, if we have the function (m=2 and n=3):

$\textbf{f}(\textbf{x}) = \begin{bmatrix}
x_{1} + x_{2}\\
\sin(x_{1}) \\
\cos(x_{1}x_{2})
\end{bmatrix}$

```Python
def f(x):
    return [x[0] + x[1], sin(x[0]), cos(x[0]*x[1])]
```


We will depend on the Numpy library, and no other libraries at this stage. We will use Numpy to define the unary functions, to ensure that the functions are evaluated efficienctly. An intentional decision was made to not depend on other libraries to ensure the package had lightweight requirements. 

# Licensing


We chose the MIT license for our project. This choice was based on multiple aspects. For one we want to allow developers to develop proprietary software with our package. Therefore a copyleft license would not be suitable as it forces developers to make their source code public. With the MIT licencse, a copyright license, developers can build proprietary software. In this way our package has a practical advantage for many different types of coders.

A second consideration is the dependency on libraries. The only library we depend upon is NumPy. This library is distributed under the liberal BSD license which is a copyright license. This means that we cannot disitrbute our own package under a copyleft license. Therefore we chose the copyright MIT license. 

For these reasons we selected the MIT license. A LICENSE file is included in the root to inform the user. 

# Feedback
## Milestone 1

Background:
- Add more about the computational graph, seed vector and dual numbers in the background.
- Further add an example/function to explain each concept and show how they are useful in AD 

How to use:
- Discuss dependencies and the guidance to the user regarding them.


Implementation:
- List the operators we will overload (most specifically the unary functions).

License:
- reconsider the license taking into account dependencies such as Numpy. 